<a href="https://colab.research.google.com/github/keerthydevisetty129/ExcelR-/blob/main/aws_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 406.7/406.7 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
   ━━━━

In [2]:
import sagemaker
import boto3
import pandas as pd
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
bucket = sagemaker_session.default_bucket()
prefix = "nlp-model-demo"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:4                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:265 in __init__                     │
│                                                                                                  │
│    262 │   │   self.lambda_client = None                                                         │
│    263 │   │   self.settings = settings if settings else SessionSettings()                       │
│    264 │   │                                                                                     │
│ ❱  265 │   │   self._initialize(                                                                 │
│    266 │   │   │   boto_session=boto_session,                                                    │
│    267 │   │   │   sagemaker_client=sagemaker_client,                                            │
│    268 │   │   │   sagemaker_runtime_client=sagemaker_runtime_client,                            │
│                                                                                                  │
│ /usr/local/lib/python3.11/dist-packages/sagemaker/session.py:293 in _initialize                  │
│                                                                                                  │
│    290 │   │                                                                                     │
│    291 │   │   self._region_name = self.boto_session.region_name                                 │
│    292 │   │   if self._region_name is None:                                                     │
│ ❱  293 │   │   │   raise ValueError(                                                             │
│    294 │   │   │   │   "Must setup local AWS configuration with a region supported by SageMaker  │
│    295 │   │   │   )                                                                             │
│    296                                                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Must setup local AWS configuration with a region supported by SageMaker.

In [6]:
df = pd.read_csv("Reviews.csv")
df = df[["Text", "Score"]].dropna()
df["Sentiment"] = df["Score"].apply(lambda x: 1 if x > 3 else 0)
df = df[["Text", "Sentiment"]]
df.to_csv("processed_reviews.csv", index=False)
s3 = boto3.resource("s3")
s3.upload_file("processed_reviews.csv", f"{prefix}/data/processed_reviews.csv")
s3_train_data = f"s3://{bucket}/{prefix}/data/processed_reviews.csv"
print("Data uploaded to:" , s3_train_data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 0>:7                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 's3.ServiceResource' object has no attribute 'upload_file'

In [7]:
%%writefile train.py
import argparse
import pandas as pd
import joblib
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
import os

Writing train.py


In [10]:
def train():
  parser = argparse.ArgumentParser()
  parser.add_argument("--train_data",type=str,default=os.environ.get("SM_CHANNEL_TRAIN"))
  args = parser.parse_args()
  train_data_path = os.path.join(args.train_data, "processed_reviews.csv")
  X = df["Text"]
  Y = df["Sentiment"]

In [11]:
pipeline = Pipeline([("tfidf",(stop_words="english")),
                     ("clf",LogisticRegression())
                     ])
pipeline.fit(X_train,y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test,y_pred))


╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ pipeline = Pipeline([("tfidf",(stop_words="english")),                                           │
│                                ▲                                                                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='?